In [1]:
import numpy as np
import glob
import pypianoroll as ppr
import time
import music21
import os
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from utils.utilsPreprocessing import *
#np.set_printoptions(threshold=np.inf)
#torch.set_printoptions(threshold=50000)

In [2]:
# ##########HYPERPARAMS#####################
epochs = 1000
learning_rate = 1e-3
weight_decay = 0.99
batch_size = 1 #CHANGE THIS VERWRIRUNG
seq_length = 8
log_interval = 1 #Log/show loss per batch
input_size=100
##########################################
##########################################

In [3]:
def createDataset(dataset, seq_length = 8, shuffle=True):    
    inputs = []
    gts = []
    data = []
    
    input_curr = []
    gt_current = []
    
    for k in range(0, len(dataset)-seq_length):
        data.append(dataset[k:k+seq_length])
    
    # permutation
    if(shuffle):
        np.random.seed(345)
        per = np.random.permutation(len(data))
        data = [data[i] for i in per]

    return np.array(data)

In [4]:
data = np.load('/media/EXTHD/niciData/embeddingDataset/YamahaPC2002NoTransposeEmbeddings.npz')
midiDatasetTrain = data['train']
midiDatasetTest = data['test']
data.close()

midiDatasetTrain = createDataset(midiDatasetTrain, seq_length = seq_length)
midiDatasetTest = createDataset(midiDatasetTest, seq_length = seq_length)

print(midiDatasetTrain.shape)
print(midiDatasetTest.shape)

(39774, 8, 100)
(9683, 8, 100)


# CDVAE

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
midiDatasetTrain = midiDatasetTrain[0:4,:,0:input_size]
print(midiDatasetTrain.shape)
midiDatasetTrain = torch.from_numpy(midiDatasetTrain)
trainLoader = torch.utils.data.DataLoader(midiDatasetTrain, batch_size=batch_size, shuffle=False, drop_last=True)

midiDatasetTest = midiDatasetTest[0:4,:,0:10]
midiDatasetTest = torch.from_numpy(midiDatasetTest)
testLoader = torch.utils.data.DataLoader(midiDatasetTest, batch_size=batch_size, shuffle=False, drop_last=True)

(4, 8, 100)


In [7]:
class LSTM(nn.Module):
    def __init__(self, batch_size=7, lstm_layers=3, hidden_size=400, seq_length=7, input_size=100):
        super(LSTM, self).__init__()
        
        self.batch_size = batch_size
        self.hidden_size = hidden_size
        self.lstm_layers = lstm_layers
        self.seq_length = seq_length
        self.input_size = input_size
        
        ###LSTM###########
        self.embedding = nn.Embedding(self.input_size, self.hidden_size)
        self.lstm = nn.LSTM(input_size=self.hidden_size, hidden_size=self.hidden_size,
                            num_layers=self.lstm_layers, batch_first=True, dropout=0)
        ##################
        
        ###LSTMCells######
        #self.lstmC1 = nn.LSTMCell(input_size=self.input_size, hidden_size=self.hidden_size, bias=True)
        #self.drop1 = nn.Dropout(p=0.1)
        #self.lstmC2 = nn.LSTMCell(input_size=self.hidden_size, hidden_size=self.hidden_size, bias=True)
        #self.drop2 = nn.Dropout(p=0.2)
        #self.lstmC3 = nn.LSTMCell(input_size=self.hidden_size, hidden_size=self.hidden_size, bias=True)
        #################
        
        self.fc = nn.Linear(self.hidden_size,self.input_size)
    
    def hiddenInitLSTM(self):
        hiddenState = torch.zeros(self.lstm_layers,
                                  self.batch_size, 
                                  self.hidden_size).double().to(device)
        cellState = torch.zeros(self.lstm_layers,
                                self.batch_size, 
                                self.hidden_size).double().to(device)
        return hiddenState, cellState
    
    def hiddenInitLSTMCell(self):
        hiddenState = torch.zeros(self.batch_size,
                                    self.hidden_size, 
                                    dtype=torch.double).to(device)
        cellState = torch.zeros(self.batch_size,
                                    self.hidden_size, 
                                    dtype=torch.double).to(device)
        hS2 = torch.zeros(self.batch_size,
                                    self.hidden_size, 
                                    dtype=torch.double).to(device)
        cS2 = torch.zeros(self.batch_size, self.hidden_size, 
                                    dtype=torch.double).to(device)
        hS3 = torch.zeros(self.batch_size, self.hidden_size, 
                                    dtype=torch.double).to(device)
        cS3 = torch.zeros(self.batch_size, self.hidden_size, 
                                    dtype=torch.double).to(device)
        
        return (hiddenState,cellState),(hS2,cS2),(hS3, cS3)

    def forward(self, embed, future = 0):#, lenghts):
         
        (h_t0, c_t0) = self.hiddenInitLSTM()
        
        input = self.embedding(embed)
        print(input.size())
        output, (h_t1, c_t1) = self.lstm(input, (h_t0, c_t0))
        print('output',output.size())
        #print('lstm weights', self.lstm.all_weights())
        output = self.fc(output[:,-1,:])
        
        """
        (h_t, c_t),(h2_t, c2_t),(h3_t,c3_t)= self.hiddenInitLSTMCell()

        outputs = []
        for i in range(0,self.seq_length-1):
            h_t, c_t = self.lstmC1(embed[:,i,:],(h_t, c_t))
            h2_t, c2_t = self.lstmC2(h_t,(h2_t, c2_t))
            h3_t, c3_t = self.lstmC3(h2_t,(h3_t, c3_t))
            output = self.fc(h3_t)
            #outputs += [output]
            
        
        for i in range(future):
            h_t, c_t = self.lstmC1(output,(h_t, c_t))
            h2_t, c2_t = self.lstmC2(h_t, (h2_t, c2_t))
            h3_t, c3_t = self.lstmC3(h2_t,(h3_t, c3_t))
            output = self.fc(h3_t)
            #outputs += [output]
        
        #outputs = torch.stack(outputs, 1)
        """
        return embed, output

    

model = LSTM(batch_size=batch_size, seq_length=seq_length, input_size=input_size).double().to(device)

#optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
optimizer = optim.RMSprop(model.parameters(),lr=learning_rate, 
                          weight_decay=weight_decay, momentum=0.9)


def train(epoch):
    model.train()
    trainLoss = 0
    criterion = nn.MSELoss()
    for batch_idx, data in enumerate(trainLoader):
        optimizer.zero_grad()
        embedding = data.long().to(device)*10
        
        #Normalize
        #embedMax = torch.max(embedding)
        #embedding /= embedMax
        
        gTruth = embedding[:,-1,:]
        inputLSTM = embedding[:,:-1,:]
        
        _ , lstmOut = model(inputLSTM, future = 0)
        
        loss = criterion(lstmOut, gTruth)
        
        loss.backward()
        trainLoss += loss.item()
        
        optimizer.step()
        if(batch_idx % log_interval == 0):
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(trainLoader.dataset),
                100. * batch_idx / len(trainLoader),
                loss.item()))# / (len(data)-(model.batch_size/model.seq_length))))
        print('embedding');print(embedding.size());print(gTruth[:,0:10])
        print('lstmOut');print(lstmOut.size());print(lstmOut[:,0:10])
    
    print('====> Epoch: {} Average Loss: {:.4f}'.format(
          epoch, trainLoss))# / (len(trainLoader.dataset)-batch_idx*(model.batch_size/model.seq_length))))
    print('\n\n')
    return trainLoss/(batch_idx+1)
    
def test(epoch):
    model.eval()
    testLoss = 0
    with torch.no_grad():
        for i, data in enumerate(testLoader):
            embedding = data.double().to(device)
            gTruthTest = embedding[:,-1,:]
            _, lstmOut = model(embedding)

            testLoss += loss_function(embedding, lstmOut).item()

    testLoss /= (len(testLoader.dataset)-i*(model.batch_size/model.seq_length))

    print('====> Test set Loss: {:.4f}'.format(testLoss))

In [8]:
import matplotlib
#%matplotlib inline
import matplotlib.pyplot as plt

trainLosses = []
testLosses = []
for epoch in range(1, epochs + 1):
    trainLosses.append(train(epoch))
    #testLosses.append(test(epoch))
    
plt.plot(trainLosses)
plt.savefig('LSTM.png')

torch.Size([1, 7, 100, 400])


RuntimeError: input must have 3 dimensions, got 4

In [ ]:
#torch.save(model,'/media/EXTHD/niciData/models/LSTM_YamahaPianoComp2002Big_1Epoch_ReLUTEST.model')

In [ ]:
print(model.parameters())